If you're viewing this on GitHub: <br />
Unfortunately the Altair/VEGA charts don't render in this preview. <br />
Please head over to this website to view the results: https://walterra.github.io/covid-19/

In [1]:
import altair as alt
import json
import pandas as pd

In [2]:
df = pd.read_csv("../data/corin_data.csv")
df['dataTime'] = pd.to_datetime(df['dataTime'], format='%d.%m.%Y %H:%M:%S')
df = df[['dataTime', 'total_confirmed', 'total_death', 'total_recovered']]
df.tail()

,dataTime,total_confirmed,total_death,total_recovered
94,2020-04-12 09:30:00,13945,350,6987.0
95,2020-04-12 09:30:00,13945,350,6987.0
96,2020-04-12 15:00:00,13894,350,6987.0
97,2020-04-13 08:00:00,13942,368,7343.0
98,2020-04-13 15:00:00,13999,368,7343.0


In [3]:
counties = ['B', 'K', 'Noe', 'Ooe', 'S', 'St', 'T', 'V', 'W']

In [4]:
df_by_day = df

In [5]:
def c_line(att='total_confirmed',col='blue',scaleType='linear'):
    return alt.Chart(df, width=300, height=200).transform_filter(
        alt.datum[att] > 0  
    ).transform_timeunit(
        date='yearmonthdate(dataTime)'
    ).transform_aggregate(
        max_att='max('+att+')',
        groupby=['date']
    ).mark_line(
        color=col,
        size=2
    ).encode(
        alt.X('date:T', title=''),
        alt.Y('max_att:Q', scale=alt.Scale(type=scaleType, base=10), title=''),
        tooltip=['date:T', 'max_att:Q']
    )

confirmed_cases = c_line('total_confirmed', '#1f77b4')
deaths = c_line('total_death', '#d62728')
recovered = c_line('total_recovered', '#ff7f0e')

confirmed_cases_log = c_line('total_confirmed', '#1f77b4', 'log')
deaths_log = c_line('total_death', '#d62728', 'log')
recovered_log = c_line('total_recovered', '#ff7f0e', 'log')

linear_scale = alt.layer(confirmed_cases, deaths, recovered).properties(title='Linear Scale')
log_scale = alt.layer(confirmed_cases_log, deaths_log, recovered_log).properties(title='Log Scale')

chart_1 = (linear_scale | log_scale).resolve_scale(color='independent')
chart_1

alt.HConcatChart(...)

In [6]:
day = alt.Chart(df, width=300, height=200
).mark_line(
    color='#1f77b4',
    size=2
).transform_timeunit(
    date='yearmonthdate(dataTime)'
).transform_aggregate(
    max_total_confirmed='max(total_confirmed)',
    groupby=['date']
).transform_window(
    rolling_max='max(max_total_confirmed)',
    rolling_min='min(max_total_confirmed)',
    frame=[-1, 0]
).transform_calculate(
    diff='datum.rolling_max - datum.rolling_min'
).encode(
    alt.X('date:T', title=''),
    alt.Y('diff:Q', title=''),
    tooltip=['date:T', 'diff:Q']
)

ma = alt.Chart(df, width=300, height=200
).mark_line(
    color='lightblue',
    size=2
).transform_timeunit(
    date='yearmonthdate(dataTime)'
).transform_aggregate(
    max_total_confirmed='max(total_confirmed)',
    groupby=['date']
).transform_window(
    rolling_max='max(max_total_confirmed)',
    rolling_min='min(max_total_confirmed)',
    frame=[-1, 0]
).transform_calculate(
    diff='datum.rolling_max - datum.rolling_min'
).transform_window(
    rolling_mean='mean(diff)',
    frame=[-6, 0]
).encode(
    alt.X('date:T', title=''),
    alt.Y('rolling_mean:Q', title=''),
    tooltip=['date:T', 'rolling_mean:Q']
)

new_cases_by_day = (ma + day).properties(title='Neue Fälle pro Tag')

day_percent = alt.Chart(df, width=300, height=200
).mark_line(
    color='#1f77b4',
    size=2
).transform_timeunit(
    date='yearmonthdate(dataTime)'
).transform_aggregate(
    max_total_confirmed='max(total_confirmed)',
    groupby=['date']
).transform_window(
    rolling_max='max(max_total_confirmed)',
    rolling_min='min(max_total_confirmed)',
    frame=[-1, 0]
).transform_calculate(
    diff='(datum.rolling_max - datum.rolling_min) / datum.rolling_max * 100'
).encode(
    alt.X('date:T', title=''),
    alt.Y('diff:Q', title=''),
    tooltip=['date:T', 'diff:Q']
)

ma_percent = alt.Chart(df, width=300, height=200
).mark_line(
    color='lightblue',
    size=2
).transform_timeunit(
    date='yearmonthdate(dataTime)'
).transform_aggregate(
    max_total_confirmed='max(total_confirmed)',
    groupby=['date']
).transform_window(
    rolling_max='max(max_total_confirmed)',
    rolling_min='min(max_total_confirmed)',
    frame=[-1, 0]
).transform_calculate(
    diff='(datum.rolling_max - datum.rolling_min) / datum.rolling_max * 100'
).transform_window(
    rolling_mean='mean(diff)',
    frame=[-6, 0]
).encode(
    alt.X('date:T', title=''),
    alt.Y('rolling_mean:Q', title=''),
    tooltip=['date:T', 'rolling_mean:Q']
)

new_cases_percent_by_day = (ma_percent + day_percent).properties(title='Neue Fälle pro Tag Steigerung %')

chart_2 = new_cases_by_day | new_cases_percent_by_day
chart_2

alt.HConcatChart(...)

In [7]:
# save chart_1 as formatted JSON
with open('../data/chart_1.json', 'w') as jsonFile:
    json.dump(json.loads(chart_1.to_json()), jsonFile, indent=4, sort_keys=True)

In [8]:
# save chart_2 as formatted JSON
with open('../data/chart_2.json', 'w') as jsonFile:
    json.dump(json.loads(chart_2.to_json()), jsonFile, indent=4, sort_keys=True)